## Численная оптимизация в логистических задачах: ДЗ

Совместим сразу два ДЗ в одно. Пусть есть классическая траспортная задача, которую надо решить двумя способами.

## 0. Генерация задания

In [91]:
import numpy as np

Зададим случайное число складов и магазинов:

In [92]:
N, M = np.random.randint(10, 100, 2)
N, M

(61, 97)

Заполним случайно массивы с количеством товара на складах и запросами магазинов:

**Кстати!** Вы же помните, что суммарное число запасов должно совпадать с суммарными запросами?

In [93]:
storages = np.zeros(N)
demands = np.zeros(M)

In [94]:
import math 

SUM = 100

def get_random_array(size: int, arr_sum: int):
    rand_n = [ np.random.random_sample() for _ in range(size) ]
    result = [ math.floor(i * arr_sum / sum(rand_n)) for i in rand_n ] 
    for _ in range(arr_sum - sum(result)): 
        result[np.random.randint(0, size-1)] += 1
    return np.array(result)

storages = get_random_array(N, SUM)
demands = get_random_array(M, SUM)

assert len(storages) == N
assert len(demands) == M
assert sum(demands) == sum(storages)

In [95]:
storages

array([1, 0, 0, 4, 0, 4, 2, 4, 1, 2, 3, 0, 2, 3, 4, 2, 2, 3, 3, 2, 1, 1,
       0, 0, 2, 1, 1, 2, 1, 2, 0, 0, 0, 5, 3, 2, 2, 0, 4, 2, 1, 2, 1, 4,
       1, 0, 1, 1, 3, 1, 0, 0, 1, 1, 3, 1, 0, 3, 0, 2, 3])

In [96]:
demands

array([1, 1, 2, 2, 2, 1, 1, 1, 3, 0, 0, 1, 1, 0, 1, 2, 0, 0, 2, 0, 1, 1,
       2, 0, 0, 1, 0, 1, 1, 3, 0, 1, 1, 2, 2, 2, 0, 1, 1, 1, 1, 1, 1, 0,
       1, 0, 1, 1, 2, 1, 3, 2, 2, 0, 1, 1, 2, 0, 1, 1, 0, 2, 2, 1, 1, 3,
       1, 1, 1, 0, 0, 0, 3, 0, 1, 0, 2, 2, 1, 1, 1, 1, 2, 0, 1, 0, 1, 1,
       1, 0, 0, 0, 2, 0, 1, 3, 0])

Сгенерируем случайно матрицу стоимости перевозок:

In [97]:
costs = np.random.randint(1, 20, size=(N, M))


In [98]:
costs

array([[ 3,  1, 17, ..., 11,  4,  2],
       [16, 10, 10, ...,  4, 11, 15],
       [ 5, 11,  4, ..., 13,  5, 11],
       ...,
       [ 4, 12,  7, ...,  9, 13,  1],
       [ 7, 13, 15, ..., 14, 16,  9],
       [ 8, 16,  6, ..., 19,  6,  9]])

Фух. Вроде все, можно приступать!

## 1. Постановка ЛП

Напишем честную постановку задачи Линейного программирования (ЛП) и попробуем решить задачу.

In [99]:
from pulp import *
from time import time

In [100]:
def make_problem(storages, demands, costs):
    prob = LpProblem("prob", LpMinimize)

    # выбор пути
    routes = LpVariable.dicts(
        name = "Routes",
        indices = ((i, j) for i in range(len(storages)) for j in range(len(demands))),
        cat = LpBinary
    )

    # целевая функция
    prob += lpSum(costs[(i, j)] * routes[(i,j)] for i in range(len(storages)) for j in range(len(demands)))

    # ограничение на спрос
    for j in range(len(demands)):
        prob += lpSum(routes[(i, j)] for i in range(len(storages))) >= demands[j]

    # ограничение на наличие
    for i in range(len(storages)):
        prob += lpSum(routes[(i, j)] for j in range(len(demands))) <= storages[i] 
    
    return prob

In [101]:
problem = make_problem(storages, demands, costs)

st_time = time()
problem.solve()
print(f"Решение заняло {time() - st_time} сек")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/diemvs/anaconda3/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/n1/x_mxj5qx7mb8bzj93wb8vzl00000gn/T/956f254ac6ba489eb30a988ac87c4dc9-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/n1/x_mxj5qx7mb8bzj93wb8vzl00000gn/T/956f254ac6ba489eb30a988ac87c4dc9-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 163 COLUMNS
At line 29749 RHS
At line 29908 BOUNDS
At line 35826 ENDATA
Problem MODEL has 158 rows, 5917 columns and 11834 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 128 - 0.01 seconds
Cgl0002I 1455 variables fixed
Cgl0004I processed model has 115 rows, 3174 columns (3174 integer (3174 of which binary)) and 6348 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I

In [102]:
LpStatus[problem.status]

'Optimal'

Посмотрим на значение целевой функции:

In [103]:
problem.objective

3*Routes_(0,_0) + 1*Routes_(0,_1) + 8*Routes_(0,_10) + 1*Routes_(0,_11) + 9*Routes_(0,_12) + 5*Routes_(0,_13) + 10*Routes_(0,_14) + 3*Routes_(0,_15) + 13*Routes_(0,_16) + 15*Routes_(0,_17) + 15*Routes_(0,_18) + 8*Routes_(0,_19) + 17*Routes_(0,_2) + 2*Routes_(0,_20) + 12*Routes_(0,_21) + 6*Routes_(0,_22) + 7*Routes_(0,_23) + 4*Routes_(0,_24) + 18*Routes_(0,_25) + 19*Routes_(0,_26) + 19*Routes_(0,_27) + 7*Routes_(0,_28) + 16*Routes_(0,_29) + 1*Routes_(0,_3) + 2*Routes_(0,_30) + 9*Routes_(0,_31) + 11*Routes_(0,_32) + 5*Routes_(0,_33) + 14*Routes_(0,_34) + 18*Routes_(0,_35) + 17*Routes_(0,_36) + 17*Routes_(0,_37) + 6*Routes_(0,_38) + 7*Routes_(0,_39) + 8*Routes_(0,_4) + 16*Routes_(0,_40) + 19*Routes_(0,_41) + 4*Routes_(0,_42) + 11*Routes_(0,_43) + 13*Routes_(0,_44) + 10*Routes_(0,_45) + 4*Routes_(0,_46) + 5*Routes_(0,_47) + 8*Routes_(0,_48) + 15*Routes_(0,_49) + 10*Routes_(0,_5) + 7*Routes_(0,_50) + 15*Routes_(0,_51) + 19*Routes_(0,_52) + 15*Routes_(0,_53) + 3*Routes_(0,_54) + 1*Routes_(0,

## 2. Метод потенциалов

А теперь давайте для тех же данных реализуем метод потенциалов: будет ли он работать лучше / быстрее?

**Кстати**: для построения начальной конфигурации используйте метод северо-западного угла.

In [104]:
# метод северно-западного угла
def get_init_configuration(d, s):
    N, M = len(s), len(d)
    result = np.zeros((N, M))
    i, j = 0, 0
    
    while i < N and j < M:
        current = min(s[i], d[j])
        result[i, j] = current
        s[i] -= current
        d[j] -= current
        
        if s[i] == 0:
            i += 1
        if d[j] == 0:
            j += 1
            
    return result

storages_test = [10, 20, 30]
demands_test = [15, 20, 25]

expected = np.array([   
    [10,  0,  0],
    [ 5, 15,  0],
    [ 0,  5, 25],
])

assert (get_init_configuration(demands_test, storages_test) == expected).all()

In [105]:
def find_potentials(basic_plan, costs):
    N, M = basic_plan.shape
    u = np.full(N, np.nan)
    v = np.full(M, np.nan)
    u[0] = 0

    while np.any(np.isnan(u)) or np.any(np.isnan(v)):
        for i in range(N):
            for j in range(M):
                if basic_plan[i, j] > 0:  
                    if np.isnan(u[i]) and not np.isnan(v[j]):
                        u[i] = costs[i, j] - v[j]
                    elif not np.isnan(u[i]) and np.isnan(v[j]):
                        v[j] = costs[i, j] - u[i]
    
    return u, v

In [106]:
def check_optimality(basic_plan, u, v, costs):
    N, M = basic_plan.shape
    delta = np.zeros((N, M))  
    
    for i in range(N):
        for j in range(M):
            if basic_plan[i, j] == 0:  
                delta[i, j] = costs[i, j] - (u[i] + v[j])
    

    if (delta >= 0).all():
        return basic_plan, True
    
    for _ in range(10):  
        min_delta_index = np.unravel_index(np.argmin(delta), delta.shape)
        i, j = min_delta_index
        if delta[i, j] >= 0:
            break
    
    return basic_plan, False

**Кстати**: для решения системы уравнений можно использовать фукнкцию np.linalg.solve(A, b), где A - квадратная матрица коэффициентов при переменных в уравнениях, а b - столбец правых частей уравнений. 

In [107]:
def solve_potentials(storages, demands, costs):
    init = get_init_configuration(d=demands.copy(), s=storages.copy())
    
    optimal = False

    u, v = find_potentials(costs, init)
        
    while optimal == False:

        delta, optimal = check_optimality(init, u, v, costs)
        if np.all(delta < 0):  
            u, v = find_potentials(costs, init) 

    return init


In [108]:
st_time = time()

res = solve_potentials(storages, demands, costs)

print(f"Решение заняло {time() - st_time} сек")

Решение заняло 0.01785111427307129 сек


Какие выводы можно сделать?

// your ideas here